## 连接数据库

In [1]:
from sqlalchemy import create_engine

# 初始化数据库连接
host = "ea-tariff-db.cgnuriesxcg4.rds.cn-northwest-1.amazonaws.com.cn"
port = "3306"
database = "wx_tc_database"
user = "admin"
password = "zenergyai"
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

## 查询原始数据

In [2]:
import pandas as pd

df = pd.read_sql_query(
    "select datetime as ts, valleye_sum_chg as chg, peake2_sum_dhg as dhg from tc1_electric_quantity_day;",
    engine)
df = df[(df["chg"] >= 0) & (df["dhg"] >= 0)]

## 衰减信息

In [ ]:
import numpy as np

df.set_index('ts', inplace=True)  # 设置索引
df.sort_index(inplace=True)  # 按时间排序
df["capacity"] = np.sqrt(df["chg"] * df["dhg"])  # 添加等效容量列
df

In [ ]:
initial = df.head(30).sort_values(by="capacity", ascending=False).iloc[0]
current = df.tail(30).sort_values(by="capacity", ascending=False).iloc[0]
attenuation = round((1 - current["capacity"] / initial["capacity"]) * 100, 2)
print(f"初始容量：{initial['capacity']}，出现时间：{initial.name}")
print(f"当前容量：{current['capacity']}，出现时间：{current.name}")
print(f"衰减率：{attenuation}%")

## 循环次数

In [ ]:
running_days = len(df)
print(f"运行天数：{running_days}天")
loop_times = running_days * 2
print(f"循环次数：{loop_times}次")

## 绘制图表

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.update_layout(title=f"项目等效容量  衰减率：{attenuation}%，运行天数：{running_days}天，循环次数：{loop_times}次")
fig.add_trace(go.Scatter(x=df.index, y=df["capacity"], name="等效容量", mode='markers'))
fig.add_trace(go.Scatter(
    x=[initial.name, current.name], y=[initial["capacity"], current["capacity"]], name="衰减率",
    mode="lines+markers", marker=dict(color="orange", size=10)))
fig.show()